In [ ]:
pip install maxminddb-geolite2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
from geolite2 import geolite2
import geoip2.database
from geoip2.errors import AddressNotFoundError

In [2]:
# Load the dataset
data_df = pd.read_csv("./Resources/bids.csv") # Jupyter Notebook
# data_df = pd.read_csv("bids.csv") # Google colab
print(data_df.shape)
data_df.head()

(7656334, 9)


,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


### Cleaning missing country

In [3]:
# Extract data that have no country info
missing_country = data_df[data_df.isnull().any(axis=1)]
print(missing_country.shape)
missing_country.head()

(8859, 9)


,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
1647,1647,dbd340def489066ae13ffa5cd68e6d2917li7,38v0a,jewelry,phone408,9759252578947368,NaN,115.235.196.83,50rtdzaq69sxgi8
1845,1845,dbd340def489066ae13ffa5cd68e6d2917li7,7msk4,jewelry,phone45,9759253684210526,NaN,161.65.211.149,3w7pkfdhqvduki1
1968,1968,ceb6c86354e97d4d849ef5b1364bb0964nxe3,jefix,sporting goods,phone150,9759254210526315,NaN,117.34.233.29,1a1l994pff8hwc4
1982,1982,dbd340def489066ae13ffa5cd68e6d2917li7,uo3lj,jewelry,phone408,9759254315789473,NaN,115.235.196.83,g0k9f3jstsx5pud
2911,2911,cdf76f38b6f46a3bd396a99123d26909xq5c3,jefix,mobile,phone150,9759259526315789,NaN,44.203.207.204,cdr9pz0wkzokbp8


Try to obtain country from the ip

In [18]:
ip_df = missing_country[["bid_id", "ip"]]
print(ip_df.shape)
ip_df.head()

(8859, 2)


,bid_id,ip
1647,1647,115.235.196.83
1845,1845,161.65.211.149
1968,1968,117.34.233.29
1982,1982,115.235.196.83
2911,2911,44.203.207.204


In [19]:
# Check the missing value of ip
ip_df.isnull().sum()

bid_id    0
ip        0
dtype: int64

Use the [MaxMind GeoIP2 Python API](https://geoip2.readthedocs.io/en/latest/) for IP information queries. Also need to download [GeoLite2-City.mmdb](https://github.com/P3TERX/GeoLite.mmdb) as data source before use.  

In [20]:
country = []
with geoip2.database.Reader('./Resources/GeoLite2-City.mmdb') as reader:
  for ip in ip_df["ip"]:
    try:
      response = reader.city(ip)
      country.append(response.country.iso_code)
    except (AddressNotFoundError, ValueError):
      response=None
      country.append("missing")

In [21]:
# Add country info to ip_df
ip_df["country"] = country
# Since the ISO 3166-1 alpha-2 codes are all capital, need to convert to lowercase to fit the original dataset
ip_df["country"] = ip_df["country"].str.lower()
print(ip_df.shape)
ip_df.head()

(8859, 3)


C:\Users\Jarvinia\AppData\Local\Temp\ipykernel_6352\1067937635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip_df["country"] = country
C:\Users\Jarvinia\AppData\Local\Temp\ipykernel_6352\1067937635.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip_df["country"] = ip_df["country"].str.lower()


,bid_id,ip,country
1647,1647,115.235.196.83,cn
1845,1845,161.65.211.149,nz
1968,1968,117.34.233.29,cn
1982,1982,115.235.196.83,cn
2911,2911,44.203.207.204,us


In [22]:
ip_df.isnull().sum()

bid_id     0
ip         0
country    5
dtype: int64

In [23]:
ip_df.fillna("missing", inplace=True)

C:\Users\Jarvinia\AppData\Local\Temp\ipykernel_6352\1092775931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip_df.fillna("missing", inplace=True)


In [24]:
ip_df.isnull().sum()

bid_id     0
ip         0
country    0
dtype: int64

In [25]:
# Replace the missing country value with ip_df
bids_cleaned = data_df.copy()
bids_cleaned.update(ip_df)
print(bids_cleaned.shape)
bids_cleaned.head()

(7656334, 9)


,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0.0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1.0,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2.0,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3.0,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4.0,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


In [26]:
# Check is there any other null values
bids_cleaned.isnull().sum()

bid_id         0
bidder_id      0
auction        0
merchandise    0
device         0
time           0
country        0
ip             0
url            0
dtype: int64

### Clean any spaces

In [27]:
# Remove any spaces
final_cleaned_df = bids_cleaned.replace({' ': ''}, regex = True) 
final_cleaned_df.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0.0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1.0,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2.0,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,homegoods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3.0,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4.0,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


In [28]:
# Save the cleaned data
# final_cleaned_df.to_csv("cleaned_bids.csv", index=False) # Google Colab
final_cleaned_df.to_csv("./Resources/cleaned_bids.csv", index=False) # Jupyter Notebook